### Data Prep

In [1]:
img_data_dir = "/media/curttigges/project-files/datasets/ms-coco/"

### Models

In [2]:
import pytorch_lightning as pl

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import TQDMProgressBar

from data.coco_data_module import COCODataModule
from q2l_labeller.pl_modules.query2label_train_module import Query2LabelTrainModule

In [3]:
pl.seed_everything(42)

Global seed set to 42


42

In [4]:
param_dict = {
    "backbone_desc":"resnest101e",
    "conv_out_dim":2048,
    "hidden_dim":256,
    "num_encoders":6,
    "num_decoders":6,
    "num_heads":8,
    "batch_size":24,
    "image_dim":576,
    "learning_rate":0.0001, 
    "momentum":0.9,
    "weight_decay":0.01, 
    "n_classes":80,
    "thresh":0.5,
    "use_cutmix":True,
    "use_pos_encoding":False,
    "loss":"BCE"
}

In [5]:
coco = COCODataModule(
    img_data_dir,
    img_size=param_dict["image_dim"],
    batch_size=param_dict["batch_size"],
    num_workers=24,
    use_cutmix=param_dict["use_cutmix"],
    cutmix_alpha=1.0)
param_dict["data"] = coco

In [6]:
pl_model = Query2LabelTrainModule(**param_dict)

In [7]:
wandb_logger = WandbLogger(project="coco-labeling")
wandb_logger.watch(pl_model, log="all")

trainer = pl.Trainer(
    max_epochs=24,
    precision=16,
    accelerator='gpu', 
    devices=1,
    logger=wandb_logger,
    callbacks=[TQDMProgressBar(refresh_rate=10)])
trainer.fit(pl_model, param_dict["data"])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: curt-tigges (ascendant). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


loading annotations into memory...
Done (t=6.53s)
creating index...
index created!
loading annotations into memory...
Done (t=4.07s)
creating index...
index created!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params
-----------------------------------------------------
0 | model          | Query2Label       | 65.8 M
1 | base_criterion | BCEWithLogitsLoss | 0     
-----------------------------------------------------
65.8 M    Trainable params
0         Non-trainable params
65.8 M    Total params
131.547   Total estimated model params size (MB)


Epoch 18:  50%|█████     | 2570/5138 [17:12:11<17:11:23, 24.10s/it, loss=0.0629, v_num=znzj, val_loss=0.0392, val_rmap=0.900, val_cat_prec=0.738, val_cat_recall=0.717, val_cat_f1=0.715, val_ovr_prec=0.863, val_ovr_recall=0.796, val_ovr_f1=0.827]  

wandb: Network error (ReadTimeout), entering retry loop.


Epoch 18:  54%|█████▎    | 2750/5138 [17:14:57<14:58:43, 22.58s/it, loss=0.0616, v_num=znzj, val_loss=0.0392, val_rmap=0.900, val_cat_prec=0.738, val_cat_recall=0.717, val_cat_f1=0.715, val_ovr_prec=0.863, val_ovr_recall=0.796, val_ovr_f1=0.827]